In [ ]:
# @title Setup & Config
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import unicodedata
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from math import e
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from collections import Counter
import ast
import random
import json
from math import isnan

plt.rc('font', family='Times New Roman', size='20')

# %config InlineBackend.figure_format='retina'


rcParams['figure.figsize'] = 24, 16


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

token = '19e8eec1bcb4772464561a9dfa2e2dd0eec9c3e89557852b31592287'
import tushare as ts

In [ ]:
ts.set_token(token)
pro = ts.pro_api()

csi300 = pro.index_daily(ts_code='399300.SZ')
csi300 = csi300.loc[('20210101' < csi300.trade_date) & (csi300.trade_date < '20211201')]
xin9 = pro.index_global(ts_code='XIN9', start_date='20210101', end_date='20211201')
new_xin9 = pd.DataFrame()
for index, row in csi300.iterrows():

    if row.trade_date in xin9.trade_date.values:
        new_xin9.loc[index, 'trade_date'] = row.trade_date
        price = xin9.loc[xin9.trade_date == row.trade_date, 'close'].values[0]
        new_xin9.loc[index, 'close'] = price
    else:
        new_xin9.loc[index, 'trade_date'] = row.trade_date
        new_xin9.loc[index, 'close'] = price
xin9 = new_xin9

csi300 = csi300.sort_index(ascending=False)
csi300 = csi300.reset_index(drop=True)
csi300.trade_date = pd.to_datetime(csi300.trade_date)
xin9 = xin9.sort_index(ascending=False)
xin9 = xin9.reset_index(drop=True)
xin9.trade_date = pd.to_datetime(xin9.trade_date)

In [ ]:

df_test = pd.read_csv("d:/data/df_all_year_srl.csv", sep='\t')

df_test = df_test.drop(
    columns=['A0', 'A1', 'verbA0A1', 'verb_mask', 'A0_mask', 'A1_mask', 'AV_num'])
df_test.SECOND_DAY = pd.to_datetime(df_test.SECOND_DAY)
df_test.day1 = pd.to_datetime(df_test.day1)
df_test.day2 = pd.to_datetime(df_test.day2)
df_test.day3 = pd.to_datetime(df_test.day3)
df_test.day4 = pd.to_datetime(df_test.day4)
df_test.day5 = pd.to_datetime(df_test.day5)

In [ ]:
df_test

In [ ]:
def string_to_tuples_list(text):
  if text is np.nan or text =='[]':
    return []
  text = text.strip('[').strip(']')
  tmp = text.split(' ')
  while '' in tmp:
    tmp.remove('')
  tmp = [*map(float,tmp)]
  return tmp

def keep_by_threshold(probability,threshold = 0.3):
    for prob in probability:
        if prob>threshold:
            return True
    return False

In [ ]:
files = ['Pretrained_RoBERT_ood.csv','Factor24_Pretrained_RoBert_SRL(FC)_V_ood.csv']

prediction_list = []
for file in files:
    prediction = pd.read_csv(file, sep='\t', index_col=None)
    prediction = prediction.drop_duplicates()
    prediction.probability = prediction.probability.apply(string_to_tuples_list)
    prediction = prediction.loc[prediction.probability.apply(keep_by_threshold)]
    prediction = prediction.reset_index(drop=True)
    
    prediction = df_test.merge(prediction, left_on='text_a', right_on='text').drop(columns=['Unnamed: 0'])
    prediction = prediction.sort_values('DATE').reset_index(drop=True)
    prediction.DATE = pd.to_datetime(prediction.DATE)
    prediction_list.append(prediction)

In [ ]:
def back_test(df, df_prediction, number):
    deposit = 1
    total_lending = 0
    total_hold = 0
    portfolio = pd.DataFrame()
    commission_fee_rate = 0.0013
    total = deposit
    days = ['SECOND_DAY', 'day1', 'day2', 'day3', 'day4', 'day5']

    price = ['original_price', 'close1', 'close2', 'close3', 'close4', 'close5']
    total_list = []
    long_weight = [0, 0, 0, 0, 0]
    short_weight = [0, 0, 0, 0, 0]
    stay_weight = [0, 0, 0, 0, 0]
    name = 'prediction'
    reverse = 0
    reverse_bool = False
    for date_idx, date in df.iterrows():
        long_weight_i = [0]
        short_weight_i = [0]
    
        df_tmp = df_prediction.loc[df_prediction.SECOND_DAY == date.trade_date]
        df_tmp = df_tmp.reset_index(drop=True)
        
        df_tmp.loc[df_tmp[name]==0,name] = 3
        df_tmp.loc[df_tmp[name]==2,name] = 0
        df_tmp.loc[df_tmp[name]==3,name] = 2

        for p_idx, p in portfolio.iterrows():
            loss_stop_sign = False
            for i in range(1, 4):
                if p[days[i]] == date.trade_date:
                    #                     print(p_idx,i)
                    if p[name] == 0:
                        total_lending = total_lending + p[price[i]] * p['share'] - p[price[i - 1]] * p['share']
                        if p[price[i]] / p[price[0]] - 1 >= 0.05:
                            short_weight_i.append(1 - p[price[i]] / p[price[0]])
                            deposit = deposit - p.share * p[price[i]] * (1 + commission_fee_rate)
                            total_lending = total_lending - p.share * p[price[i]]
                            loss_stop_sign = True
                            portfolio = portfolio.drop(p_idx)

                    elif p[name] == 2:
                        #                         print(p_idx,i)
                        total_hold = total_hold + p[price[i]] * p['share'] - p[price[i - 1]] * p['share']
                        if p[price[i]] / p[price[0]] - 1 <= -0.05:
                            long_weight_i.append(p[price[i]] / p[price[0]] - 1)
                            deposit = deposit + p.share * p[price[i]] * (1 - commission_fee_rate)
                            total_hold = total_hold - p.share * p[price[i]]
                            loss_stop_sign = True
                            portfolio = portfolio.drop(p_idx)
                    break

            if p[days[number]] == date.trade_date and (not loss_stop_sign):
                if p[name] == 0:
                    short_weight_i.append(1 - p[price[i]] / p[price[0]])
                    deposit = deposit - p.share * p[price[number]] * (1 + commission_fee_rate)
                    total_lending = total_lending - p.share * p[price[number]]
                elif p[name] == 2:
                    deposit = deposit + p.share * p[price[number]] * (1 - commission_fee_rate)
                    total_hold = total_hold - p.share * p[price[number]]
                    long_weight_i.append(p[price[i]] / p[price[0]] - 1)
                portfolio = portfolio.drop(p_idx)

        long_weight.append(np.mean(long_weight_i))
        short_weight.append(np.mean(short_weight_i))
        if long_weight[len(long_weight) - 1] < 0 and short_weight[len(short_weight) - 1] < 0:
            stay_weight.append(-long_weight[len(long_weight) - 1] / 2 - short_weight[len(short_weight) - 1] / 2)
        else:
            stay_weight.append(0)
        
        long_divider = 1
        short_divider = 1
        if len(df_tmp.loc[df_tmp[name] == 0]) != 0:
            short_divider = 1 / len(df_tmp.loc[df_tmp[name] == 0])
        if len(df_tmp.loc[df_tmp[name] == 2]) != 0:
            long_divider = 1 / len(df_tmp.loc[df_tmp[name] == 2])
        weight_st = pow(50, sum(stay_weight[-3:]))
        weight_l = pow(50, sum(long_weight[-3:]))
        weight_sh = pow(50, sum(short_weight[-3:]))
        base = deposit - total_lending
        
        if len(total_list)>=3:
            if total_list[-1]/total_list[-2]-1 <= -0.03:
                reverse_bool = not reverse_bool
                reverse = 3
            if -0.002 < total_list[-1]/total_list[-2]-1 < 0.002 and -0.002 < total_list[-2]/total_list[-3]-1 < 0.002:
                weight_st = weight_st*2
        if reverse > 0 and reverse_bool:
            reverse -= 1
            if reverse ==0 :
                reverse_bool = False

            df_tmp[name]=1
        for stock_idx, stock in df_tmp.iterrows():
            l = stock.probability[2]
            sh = stock.probability[0]
            st = stock.probability[1]
            
            share_long = weight_l*l / (weight_l*l + weight_sh*sh + weight_st*st) * (base) / stock.original_price * long_divider
            share_short = weight_sh*sh / (weight_l*l + weight_sh*sh + weight_st*st) * (base) / stock.original_price * short_divider
            
            if stock[name] == 0:
                deposit = deposit + share_short * stock.original_price
                total_lending += share_short * stock.original_price
                stock['share'] = share_short
            elif stock[name] == 2 and deposit > 0.45:
                deposit = deposit - share_long * stock.original_price
                total_hold = total_hold + share_long * stock.original_price
                stock['share'] = share_long
            else:
                continue

            portfolio = portfolio.append(stock)
        portfolio = portfolio.reset_index(drop=True)
        
        total = total_hold + deposit - total_lending
        total_list.append(total)
    return total_list

In [ ]:
day = 1

x = csi300.trade_date.values
xin = (xin9.close / xin9.close[0]).values
csi = (csi300.close / csi300.close.loc[0]).values

prediction_results = [xin, csi]
for prediction in prediction_list:
    prediction_results.append(back_test(csi300, prediction, day))

files_name = ['XIN9', 'CSI300'] + files
for index, result in enumerate(prediction_results):
    plt.plot(x, result, label=files_name[index])
plt.grid(color='grey', linestyle='--')
plt.legend()
plt.show()

In [ ]:
day = 2

x = csi300.trade_date.values
xin = (xin9.close / xin9.close[0]).values
csi = (csi300.close / csi300.close.loc[0]).values

prediction_results = [xin, csi]
for prediction in prediction_list:
    prediction_results.append(back_test(csi300, prediction, day))

files_name = ['XIN9', 'CSI300'] + files
for index, result in enumerate(prediction_results):
    plt.plot(x, result, label=files_name[index])
plt.grid(color='grey', linestyle='--')
plt.legend()
plt.show()

In [ ]:
result= [x]

In [ ]:
result.extend(prediction_results)

In [ ]:
result_df = pd.DataFrame(result).T

In [ ]:
result_df.columns = {'date','XIN9', 'CSI300', 'StockNet','HAN','Roberta-wwm-EXT', 'SRLP+SRL+Factor'}

In [ ]:
result_df.to_csv('data_for_Dr_lin.csv')

In [ ]:
files_name = ['XIN9', 'CSI300'] + files

## annualized rate of return

In [ ]:
def anual_rate(total):
    return pow(total,(250/len(x)))-1

y_b = []
for prediction in prediction_results:
    y_b.append(prediction[-1])
print('Total return: ',np.array(y_b)-1)
y_b = [*map(anual_rate,y_b)]
print('Anual return: ',y_b)
plt.bar(files_name,y_b)
plt.axhline(y_b[0], color="black", linestyle="--")
plt.xticks(rotation=45)
plt.ylabel("Annualized Rate of Return")
for a,b in zip(range(len(files_name)),y_b):
    plt.text(a+0.05,b+0.05,'%.2f' %b, ha='center',va='bottom')

## 回撤

In [ ]:
def drawdown(y,period):
    drawdown = 0
    drawdown_list = []
    max_drawdown = 0
    
    for i in range(period):
        price_tmp = y[0:i+1]
        max_price = np.max(price_tmp)
        max_index = np.argmax(price_tmp)
        min_price = np.min(price_tmp[max_index:i+1])
        if min_price/max_price-1 < drawdown:
            drawdown = min_price/max_price -1
            if drawdown< max_drawdown:
                max_drawdown = drawdown
        drawdown_list.append(drawdown)
        
    for i in range(len(y)-period):
        drawdown = 0
        price = y[i:i+period]
        for i in range(period):
            price_tmp = price[0:i+1]
            max_price = np.max(price_tmp)
            max_index = np.argmax(price_tmp)
            min_price = np.min(price_tmp[max_index:i+1])
            if min_price/max_price-1 < drawdown:
                drawdown = min_price/max_price -1
                if drawdown< max_drawdown:
                    max_drawdown = drawdown
        drawdown_list.append(drawdown)
        
    return max_drawdown, drawdown_list

period =  20

drawdown_list = []
for index,name in enumerate(files_name):
    dd = drawdown(prediction_results[index],period)
    print(name,dd[0])
    drawdown_list.append(dd[1])

In [ ]:
x
a = [x]
a.extend(drawdown_list)
drawback_df = pd.DataFrame(a).T

In [ ]:

drawback_df.columns = {'date','XIN9', 'CSI300','Roberta-wwm-EXT', 'SRLP+SRL+Factor'}
drawback_df.to_csv('drawback_for_Dr_lin.csv')

In [ ]:
drawdown_list.extend(prediction_results)

In [ ]:
for index, result in enumerate(drawdown_list):
    plt.plot(x, result)
plt.grid(color='grey', linestyle='--')
plt.legend()
plt.ylim((-2, 2))
plt.show()

## Sharp Ratio

In [ ]:
def sharp_ratio(y,period,risk_free):
    sharp_ratio_list = []
    risk_free_period = pow(1+risk_free,period/250)-1
    for i in range(period,len(y),period):
        sharp_ratio_list.append(y[i]-y[i-period])
    return(np.mean(np.array(sharp_ratio_list)-risk_free_period)/np.std(sharp_ratio_list))

period = 20
risk_free_ratio = 0.03
for index,name in enumerate(files_name):
    print(name,sharp_ratio(prediction_results[index],period,risk_free_ratio))
